# HowTo -AnglesAndInternodes- Extract fruits angles and internodes distance from a tree graph

In this notebook you will learn:

1. how to connect to a ROMI database
2. how to extract fruits sequences of successive angles and internode lengths
3. how to visualize the estimated fruit directions

This notebook **assume** that you have:
- declared the `ROMI_DB` environment variable as the path to the database directory to use
- processed the test dataset with the _geometric pipeline,_ so we can access the fileset containing the data we want to start with...

Remember, the aim of this notebook is to show you how it works "under the hood".
This is not how you should process your data, that is done thanks to the `romi_run_task` CLI tool.

In [ ]:
import os
from math import degrees

import ipywidgets as widgets
import plotly.graph_objects as go
from plant3dvision.arabidopsis import compute_stem_and_fruit_directions, \
    compute_angles_and_internodes_from_directions
from plant3dvision.visu import plotly_treegraph, plotly_sequences, plotly_pointcloud_data, plotly_direction_data
from plantdb.fsdb import FSDB
from plantdb.io import read_graph, read_point_cloud

# Connect to the database & get the initial data

If you did not declare a `ROMI_DB` environment variable, you can do it by uncommenting the next cell and setting it to the right value.

In [ ]:
# os.environ['ROMI_DB'] = "/path/to/test/data"

### Connect to the database

In [ ]:
db = FSDB(os.environ['ROMI_DB'])  # requires definition of this environment variable!
db.connect()

Once you are connected to the database, you can list the available scan *dataset* with `db.list_scans()`.

### Select a dataset

We now select a dataset (with the `Dropdown` widget) for the demo:

In [ ]:
scan_name = widgets.Dropdown(options=db.list_scans(), value=db.list_scans()[0], description='Dataset:')
display(scan_name)

In [ ]:
scan = db.get_scan(scan_name.value)

If you did not process this dataset yet, from the `plant3dvision` root directory, you can do it with:
```
romi_run_task AnglesAndInternodes $ROMI_DB/<selected_dataset> --config plant-3d-vision/configs/geom_pipe_real.toml
```

To list the available *filesets* in this *scan dataset*:

In [ ]:
scan.list_filesets()

### Get the skeleton fileset

The skeleton resulting from a _CurveSkeleton_ task is to be found in the 'CurveSkeleton*' fileset.

In [ ]:
tree_fs = scan.get_filesets(query={"task_name": "TreeGraph"})[0]
print(tree_fs.path().stem)

### Get and load the skeleton file

Once you have access to the right fileset, you may access the skeleton file and load it as follows:

In [ ]:
tree_file = tree_fs.get_files()[0]
tree = read_graph(tree_file)

### Visualize the tree graph

It is possible to visualize the tree graph with the `plotly_treegraph` method from `plant3dvision.visu` as follows:

In [ ]:
fig = plotly_treegraph(tree)

In [ ]:
fig.show()

## Computing the angle and internode sequences

It is now time to process the *tree graph* to get the sequences of successive angles and internode lengths between the fruits.

This is done in two steps:
1. compute the stem directions, the fruit directions and the branching point positions
2. compute the angle and internode sequences using the fruit directions and the branching point positions

### Compute the stem directions, the fruit directions and the branching point positions

This is done with the `compute_stem_and_fruit_directions` function from `plant3dvision.arabidopsis` as follows:

In [ ]:
fruit_dirs, stem_dirs, bp_coords, fruit_pts = compute_stem_and_fruit_directions(tree, max_node_dist=10.,
                                                                                min_fruit_length=10.)

### Visualize the estimated fruit directions and branching points

Let's represent both objects and see how good the estimations are for the fruit directions and their branching points.

Let's start by loading the point cloud to get a "reference":

In [ ]:
pcd_fs = scan.get_filesets(query={"task_name": "PointCloud"})[0]
pcd_file = pcd_fs.get_files()[0]
pcd = read_point_cloud(pcd_file)

Now we generate graph objects for point cloud and fruit directions:

In [ ]:
go_pcd = plotly_pointcloud_data(pcd, marker_kwargs={'size': 1})
go_dir = plotly_direction_data(fruit_dirs, bp_coords, 'fruit')

Finally, we use Plotly to represent all this:

In [ ]:
layout_style = dict(height=900, width=800, showlegend=False)
fig = go.Figure(data=[go_pcd] + go_dir, layout=layout_style)
fig.update_scenes(aspectmode='data')

### Compute the angle and internode sequences

This is done with the `compute_angles_and_internodes_from_directions` function from `plant3dvision.arabidopsis` as follows:

In [ ]:
sequences = compute_angles_and_internodes_from_directions(fruit_dirs, stem_dirs, bp_coords)

As the returned angles are in radians, we convert them to degrees:

In [ ]:
sequences['angles'] = list(map(degrees, sequences['angles']))

### Visualize the sequences

Once the sequences of angles and internodes has been obtained, we can visualize them with the `plotly_sequences` method from `plant3dvision.visu` as follows:

In [ ]:
fig = plotly_sequences(sequences, height=600)

In [ ]:
fig.show()

We may now **disconnect** from the database as we will not need it anymore:

In [ ]:
db.disconnect()